In [1]:
import pandas as pd
import os
from config import get_project_root

DATA_PATH= f"{get_project_root()}/data"
RAW_PATH = f'{DATA_PATH}/EFE_raw.csv'
INITIAL_POP = 44_044_811

In [2]:
def process_raw_data(path):
    raw_df = pd.read_csv(path, na_values='NN',thousands=r'.', sep=',')
    raw_df.rename(columns={'AÑO':'ANO'}, inplace=True)  # Cambiar las Enies
    raw_df.set_index('ANO', inplace=True)  # Indexar por ANO
    raw_df.dropna(how='all', inplace=True)  # Remover filas vacias
    raw_df.drop(raw_df.tail(1).index,inplace=True)  # Quitar la ultima fila de la pop total
    return raw_df


In [3]:
def extract_significant_df(raw_df, batch: int):
    df = raw_df[raw_df.columns[batch*14:(batch+1)*14]].copy(deep=True)  # Obtener 12 meses y total
    df.columns=df.iloc[0]   # Primera fila son las columnas
    df.drop(df.head(1).index,inplace=True)  # remover primera fila
    df.dropna(how='all', axis=1, inplace=True)
    df = df.apply(lambda x: x.str.replace('.', '', regex=False))
    df = df.apply(pd.to_numeric)
    return df

In [4]:
raw = process_raw_data(RAW_PATH)
raw

,Cantidad de infectados,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,...,Unnamed: 75,Unnamed: 76,Unnamed: 77,Unnamed: 78,Unnamed: 79,Unnamed: 80,Unnamed: 81,Unnamed: 82,Unnamed: 83,Unnamed: 84
ANO,,,,,,,,,,,,,,,,,,,,,
NaN,ENE,FEB,MAR,ABR,MAY,JUN,JUL,AGOS,SEP,OCT,...,MAY,JUN,JUL,AGOS,SEP,OCT,NOV,DIC,otros,Total
2018,0,0,0,0,12,0,0,4,1,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2019,0,0,0,125,125,0,0,0,26,23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2020,47,22,7,0,1,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2021,0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2022,1,0,1,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
TOTAL,48,22,8,125,138,0,0,4,27,25,...,0,0,0,0,0,0,0,0,NaN,0


## Infectados

In [5]:
# Infectados son los primeros 14

infectados = extract_significant_df(raw, batch=0)

In [6]:
infectados

nan,ENE,FEB,MAR,ABR,MAY,JUN,JUL,AGOS,SEP,OCT,NOV,DIC,otros,Total
ANO,,,,,,,,,,,,,,
2018,0,0,0,0,12,0,0.0,4.0,1.0,2.0,0.0,0.0,12,31
2019,0,0,0,125,125,0,0.0,0.0,26.0,23.0,37.0,31.0,125,492
2020,47,22,7,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,77,154
2021,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
2022,1,0,1,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,1,3
TOTAL,48,22,8,125,138,0,0.0,4.0,27.0,25.0,37.0,31.0,215,680


Parece que aca los NaN serian 0 asi se condice con los totales, los ponemos y escribimos el csv correspondiente

In [7]:
infectados.fillna(0, inplace=True)
infectados

nan,ENE,FEB,MAR,ABR,MAY,JUN,JUL,AGOS,SEP,OCT,NOV,DIC,otros,Total
ANO,,,,,,,,,,,,,,
2018,0,0,0,0,12,0,0.0,4.0,1.0,2.0,0.0,0.0,12,31
2019,0,0,0,125,125,0,0.0,0.0,26.0,23.0,37.0,31.0,125,492
2020,47,22,7,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,77,154
2021,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
2022,1,0,1,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,1,3
TOTAL,48,22,8,125,138,0,0.0,4.0,27.0,25.0,37.0,31.0,215,680


In [8]:
infectados.to_csv(f'{DATA_PATH}/infectados.csv')

## Vacunados

In [9]:
vacunados = extract_significant_df(raw, batch=1)
vacunados

nan,ENE,FEB,MAR,ABR,MAY,JUN,JUL,AGOS,SEP,OCT,NOV,DIC,Total
ANO,,,,,,,,,,,,,
2018,48076,55626,69592,76943,58997,44889,45554.0,66286.0,52629.0,778760.0,258334.0,48350.0,1604036
2019,51939,65760,67004,59395,60711,47079,51991.0,53737.0,74098.0,145010.0,99466.0,80905.0,857095
2020,15092,25081,32811,19156,21227,22352,19945.0,13777.0,6366.0,4859.0,5921.0,3899.0,190486
2021,3444,3472,3445,4383,4100,3828,3487.0,2769.0,55193.0,62399.0,81989.0,52543.0,281052
2022,88752,247518,227124,178109,136552,72294,NaN,NaN,NaN,NaN,NaN,NaN,950349
TOTAL,207303,397457,399976,337986,281587,190442,120977.0,136569.0,188286.0,991028.0,445710.0,185697.0,3883018


Validemos los Nan

In [10]:
vacunados[vacunados.columns[0:12]].sum(axis=1) == vacunados.Total

ANO
2018     True
2019     True
2020     True
2021     True
2022     True
TOTAL    True
dtype: bool

En efecto los NaN deberian ser 0

In [11]:
vacunados.fillna(0, inplace=True)
vacunados.to_csv(f'{DATA_PATH}/vacunados.csv')

## Habitantes

In [12]:
habitantes = extract_significant_df(raw, batch=2)
habitantes

nan,JUL
ANO,
2018,44494502.0
2019,44938712.0
2020,45376763.0
2021,45808747.0
2022,47327407.0
TOTAL,NaN


Removemos ultima fila

In [13]:
habitantes.drop(habitantes.tail(1).index,inplace=True)
habitantes

nan,JUL
ANO,
2018,44494502.0
2019,44938712.0
2020,45376763.0
2021,45808747.0
2022,47327407.0


In [14]:
habitantes.fillna(0, inplace=True)
habitantes.to_csv(f'{DATA_PATH}/habitantes.csv')